# Ejercicios 7 y 8

Todas las animaciones y gráficos se pueden encontrar en el [drive](https://drive.google.com/drive/folders/1Bab-1RPkFpFJQ-6-g4lxLmxXzEoLf3yT?usp=sharin).

In [13]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math as mt

const_N = 1000 # Cantidad de personas
const_proporcion_contagiados = 0.02 # Porcentaje de contagiados iniciales
const_tamanio_mapa = 200 # Tamanio del mapa
const_instantes_de_tiempo = 1000 # Instantes simulados

# Esto en "True" es el ej 8
correlated_random_walking = True

# Se puede observar que cuando se activa la opcion de correlated walking el contagio de la totalidad de las
# personas se produce considerablemente mas rapido, esto es debido a que las personas tienden por momentos
# a desplazarse mayores distancias en determinados sentidos, lo que provoca el acercamiento con otras personas
# y por lo tanto aumenta la probabilidad de contagio.

const_prob_cercana_contagio = 0.7   # celdas <= 3
const_prob_lejana_contagio = 0.5   # 3 < celdas <= 6

const_up = 0
const_down = 1
const_right = 2
const_left = 3

class Persona:

  def __init__(self, tipo_de_persona, esta_enfermo, pos_x, pos_y):
    turnos = {"A":1, "B":2, "C":4}
    self._turnos_de_espera = turnos[tipo_de_persona]
    self._esta_enfermo = esta_enfermo
    self._turnos_esperados = 0
    self._pos_x = pos_x
    self._pos_y = pos_y
    self._movimiento_anterior = None

  def intentar_moverse(self):
        self._turnos_esperados += 1

        if self._turnos_esperados == self._turnos_de_espera:
            self._turnos_esperados = 0
            return True

        return False

  def intentar_contagiar(self, probabilidad_de_contagio):
    if (not self._esta_enfermo):
      if rd.uniform(0, 1) <= probabilidad_de_contagio:
        self._esta_enfermo = True
        
  def puede_contagiar(self):
    return (self._esta_enfermo)

  def obtener_posicion(self):
    return self._pos_x, self._pos_y

  def modificar_posicion(self, x, y):
    self._pos_x = x
    self._pos_y = y
    
  def obtener_movimiento_anterior(self):
    return self._movimiento_anterior

  def modificar_ultimo_movimiento(self, movimiento):
    self._movimiento_anterior = movimiento


class Pandemia:

  def __init__(self):
    self.filas = const_tamanio_mapa
    self.columnas = const_tamanio_mapa
    self.mapa = generar_mapa(const_tamanio_mapa, const_tamanio_mapa)
    self.personas = poblar_mapa(self.mapa, const_N)
    self.lista_instantes = []
    self.lista_contagiados = []
    self.lista_sanos = []
    self.instantes_pasados = 0

  def info_personas(self):
      posicion_x = []
      posicion_y = []
      estado = []

      for persona in self.personas:
          x, y = persona.obtener_posicion()
          posicion_y.append(y)
          posicion_x.append(x)
          if persona.puede_contagiar():
              estado.append('red')
          else:
              estado.append('green')

      return posicion_x, posicion_y, estado

  def siguiente_instante(self):
    self.lista_instantes.append(self.instantes_pasados)
    contagiados, sanos = estado_personas(self.personas)
    self.lista_contagiados.append(contagiados)
    self.lista_sanos.append(sanos)

    for persona in self.personas:

        if persona.puede_contagiar():
            contagiar_cercanos(self.mapa, persona)

        if persona.intentar_moverse():
            mover_persona(self.mapa, persona)

    self.instantes_pasados += 1



  def simular(self):
      # Sacado de un ejemplo de la documentacion de animation de pyplot
      #---------------------------------------
    
      fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(10, 10))
      ax.set_title("Simulación de contagios en una pandemia")
      ax.set_xlabel('Celdas (0.5 Metros)')
      ax.set_ylabel('Celdas (0.5 Metros)')
      personas = plt.scatter([], [])
      rect = plt.Rectangle([0, 0], 200, 200, ec='none', lw=1, fc='none')
      #---------------------------------------

      def animate(i):
          nonlocal rect, fig, personas
          ax.set_title("Simulación de contagios en una pandemia ( Instante Nro " + str(self.instantes_pasados) + " )")
          self.siguiente_instante()
          ms = 50 # Tamanio de los puntos
          rect.set_edgecolor('black')
          info_personas = self.info_personas()
          personas.remove()
          personas = plt.scatter(info_personas[0], info_personas[1], c=info_personas[2], s=ms)
          return personas, rect

      # Ejecuta "const_instantes_de_tiempo" veces la funcion "animate" con la que crea cada frame de la animacion
      ani = animation.FuncAnimation(fig, animate, interval=1, blit=True, frames=const_instantes_de_tiempo, repeat=False)

      ani.save('animation.mp4', fps=15)
    
      plt.close()
      plt.plot(self.lista_instantes, self.lista_contagiados)
      plt.xlabel("Instantes")
      plt.ylabel("Numero de contagiados")
      plt.title("Contagiados en funcion del tiempo")
      plt.savefig("Contagiados en funcion del tiempo.png")
      plt.clf()
      plt.plot(self.lista_instantes, self.lista_sanos)
      plt.xlabel("Instantes")
      plt.ylabel("Numero de sanos")
      plt.title("Sanos en funcion del tiempo")
      plt.savefig("Sanos en funcion del tiempo.png")


def generar_mapa(filas, columnas):
    mapa = []
    for i in range(filas):
        mapa.append([])
        for j in range(columnas):
            mapa[i].append(None)
    return mapa

def contagiar_cercanos(mapa, persona):
    x, y = persona.obtener_posicion()
    
    for i in range(-6, 7): # Rango [-6, 6]
        for j in range(-6, 7): # Rango [6, 6]
            pos_x = x + i
            pos_y = y + j
            
            # Utilizo la distancia Manhattan (El enunciado no especifica)
            
            if (posicion_es_valida(mapa, pos_x, pos_y)) and (mapa[pos_x][pos_y] is not None):
                if (((abs(i) + abs(j)) <= 6) and (abs(i) + abs(j)) > 3):
                    mapa[pos_x][pos_y].intentar_contagiar(const_prob_lejana_contagio)
                elif (abs(i) + abs(j) <= 3):
                    mapa[pos_x][pos_y].intentar_contagiar(const_prob_cercana_contagio)
                
            
def estado_personas(lista_personas):
    contagiados = 0
    sanos = 0
    for persona in lista_personas:
        if (persona.puede_contagiar()):
            contagiados += 1
        else:
            sanos += 1
    return contagiados, sanos

def poblar_mapa(mapa, cantidad_de_gente):

    cupos_restantes = cantidad_de_gente
    cantidad_A = int(cantidad_de_gente * 0.70)
    cantidad_B = int(cantidad_de_gente * 0.25)
    cantidad_C = cantidad_de_gente - (cantidad_A + cantidad_B)
    cantidades = {"A":cantidad_A, "B":cantidad_B, "C":cantidad_C}
    return crear_personas(mapa, cantidades)

def crear_personas(mapa, cantidades):
    lista_personas = []
    contagiados_generados = 0
    cantidad_de_gente = cantidades["A"] + cantidades["B"] + cantidades["C"]
    contagiados_totales = int(cantidad_de_gente * const_proporcion_contagiados)

    probabilidad_contagiado = const_proporcion_contagiados

    for k in range(cantidad_de_gente):
        esta_contagiado = rd.uniform(0, 1) <= probabilidad_contagiado
        if (esta_contagiado):
            contagiados_generados += 1
            
        # Si ya se generaron todos los contagiados dejo de generarlos
        if (contagiados_generados == contagiados_totales):
            probabilidad_contagiado = -1
            
        # Si la cantidad de contagiados que faltan generar es igual a la cantidad de personas que faltan generar genero solo contagiados
        elif ((contagiados_totales - contagiados_generados) == (cantidad_de_gente - k - 1)):
            probabilidad_contagiado = 1
            
        pos_x, pos_y = posicion_libre(mapa)
        persona = persona_random(cantidades, esta_contagiado, pos_x, pos_y)
        
        lista_personas.append(persona)
        mapa[pos_x][pos_y] = persona
    return lista_personas
    

#Devuelve dos enteros que representan una posicion aleatoria en el mapa que no se encuentra ocupada.
def posicion_libre(mapa):
    pos_x, pos_y = rd.randint(1, len(mapa) - 1), rd.randint(1, len(mapa[0]) - 1)
    while(not (mapa[pos_x][pos_y] is None)):
        pos_x, pos_y = rd.randint(1, len(mapa) - 1), rd.randint(1, len(mapa[0]) - 1)
    return pos_x, pos_y

def persona_random(cantidades, esta_enferma, pos_x, pos_y):
    cantidad_actual = cantidades["A"] + cantidades["B"] + cantidades["C"]
    rand_num = rd.randint(1, cantidad_actual)
    tipo = 0

    if (rand_num <= cantidades["A"]):
        tipo = "A"
    elif ((rand_num > cantidades["A"]) and (rand_num <= cantidades["A"] + cantidades["B"])):
        tipo = "B"
    else:
        tipo = "C"

    cantidades[tipo] -= 1
    return Persona(tipo, esta_enferma, pos_x, pos_y)

#Verifica si la posicion esta libre (No hay ninguna persona en esa posicion)
def posicion_esta_libre(mapa, pos_x, pos_y):
    return mapa[pos_x][pos_y] is None

#Verifica si la posicion es valida (Si la posicion se encuentra en el rango de valores que conforman los limites de la mapa).
def posicion_es_valida(mapa, pos_x, pos_y):
    return (pos_x >= 0) and (pos_y >= 0) and (pos_x < len(mapa)) and (pos_y < len(mapa[0]))

def repetir_movimiento(mapa, persona, pos_x, pos_y):
    se_movio = False
    
    if (persona.obtener_movimiento_anterior() == const_up):
        if ((posicion_es_valida(mapa, pos_x, pos_y + 1)) and (posicion_esta_libre(mapa, pos_x, pos_y + 1))):
            persona.modificar_posicion(pos_x, pos_y + 1)
            se_movio = True
        
    elif (persona.obtener_movimiento_anterior() == const_down):
        if ((posicion_es_valida(mapa, pos_x, pos_y - 1)) and (posicion_esta_libre(mapa, pos_x, pos_y - 1))):
            persona.modificar_posicion(pos_x, pos_y - 1)
            se_movio = True
        
    elif (persona.obtener_movimiento_anterior() == const_left):
        if ((posicion_es_valida(mapa, pos_x - 1, pos_y)) and (posicion_esta_libre(mapa, pos_x - 1, pos_y))):
            persona.modificar_posicion(pos_x - 1, pos_y)
            se_movio = True
            
    elif (persona.obtener_movimiento_anterior() == const_right):
        if ((posicion_es_valida(mapa, pos_x + 1, pos_y)) and (posicion_esta_libre(mapa, pos_x + 1, pos_y))):
            persona.modificar_posicion(pos_x + 1, pos_y)
            se_movio = True

    return se_movio

    
def mover_persona(mapa, persona):
    pos_x, pos_y = persona.obtener_posicion()
    se_movio = False
    
    
    # Con probabilidad 0.4 se repite el movimiento anterior
    if ((correlated_random_walking == True) and (rd.uniform(0, 1) < 0.4) and (persona.obtener_movimiento_anterior() is not None)) : # Repito direccion y sentido
        se_movio = repetir_movimiento(mapa, persona, pos_x, pos_y)
        
    #La probabilidad de moverse en direccion horizontal o verticalmente es del 0.5 para cada caso.
    #De igual manera, la probabilidad de moverse en cada sentido es de 0.5 para cada caso.
    #No se permiten movimientos en diagonal.
    
    # Obtengo un nuevo movimiento aleatorio (Puede repetirse)
    elif rd.uniform(0, 1) < 0.5: #Movimiento Horizontal

        if (rd.uniform(0, 1) < 0.5) and (posicion_es_valida(mapa, pos_x, pos_y + 1)) and (posicion_esta_libre(mapa, pos_x, pos_y + 1)):

            persona.modificar_posicion(pos_x, pos_y + 1)
            persona.modificar_ultimo_movimiento(const_up)
            se_movio = True
            

        elif (posicion_es_valida(mapa, pos_x, pos_y - 1)) and (posicion_esta_libre(mapa, pos_x, pos_y - 1)):

            persona.modificar_posicion(pos_x, pos_y - 1)
            persona.modificar_ultimo_movimiento(const_down)
            se_movio = True

    else: #Movimiento Vertical

        if (rd.uniform(0, 1) < 0.5) and (posicion_es_valida(mapa, pos_x - 1, pos_y)) and (posicion_esta_libre(mapa, pos_x - 1, pos_y)):

            persona.modificar_posicion(pos_x - 1, pos_y)
            persona.modificar_ultimo_movimiento(const_left)
            se_movio = True

        elif (posicion_es_valida(mapa, pos_x + 1, pos_y)) and (posicion_esta_libre(mapa, pos_x + 1, pos_y)):

            persona.modificar_posicion(pos_x + 1, pos_y)
            persona.modificar_ultimo_movimiento(const_right)
            se_movio = True

    if se_movio:
        x, y = persona.obtener_posicion()
        mapa[x][y] = persona
        mapa[pos_x][pos_y] = None
        

#### Ejercicio 7
Calculamos la cantidad media de iteraciones hasta que la población entera se contagie:

In [24]:
correlated_random_walking = False

In [25]:
import numpy as np

def iteraciones_hasta_contagiarse_total():
    pand = Pandemia()
    todos_contagiados = False
    i = 0
    while not todos_contagiados:
        pand.siguiente_instante()
        pos_x, pos_y, estado = pand.info_personas()
        i += 1
        todos_contagiados = all(elem == 'red' for elem in estado)
    
    return i

iter = []
for x in range(20):
    iter.append(iteraciones_hasta_contagiarse_total())

print("Cantidad media de iteraciones hasta contagio total: %.3f" % np.mean(iter))

Cantidad media de iteraciones hasta contagio total: 343.450


Veamos como cambia este valor cuando varía la cantidad de personas en el área: 

In [26]:
for i in range(3):
    const_N = 250 * (i+1)
    iter = []
    for x in range(10):
        iter.append(iteraciones_hasta_contagiarse_total())

    print("Cantidad media de iteraciones hasta contagio total con %d personas: %.3f" % (const_N, np.mean(iter)))

Cantidad media de iteraciones hasta contagio total con 250 personas: 2253.700
Cantidad media de iteraciones hasta contagio total con 500 personas: 759.100
Cantidad media de iteraciones hasta contagio total con 750 personas: 402.500


Se puede observar claramente que a medida que aumentamos el valor de N la poblacion se contagia por completo mas rapido. Lo cual es logico ya que las personas estan mas proximas entre si.

#### Ejercicio 8
Para este ejercicio modificamos el comportamiento de las personas en nuestro modelo. Ahora perfieren continuar moviendose en la misma dirección que venían caminando en vez de que sea totalmente aleatorio. Veamos como se modificaron los resultados del ejercicio 7:

In [27]:
correlated_random_walking = True

In [28]:
for i in range(4):
    const_N = 250 * (i+1)
    iter = []
    for x in range(10):
        iter.append(iteraciones_hasta_contagiarse_total())

    print("Cantidad media de iteraciones hasta contagio total con %d personas: %.3f" % (const_N, np.mean(iter)))

Cantidad media de iteraciones hasta contagio total con 250 personas: 1057.600
Cantidad media de iteraciones hasta contagio total con 500 personas: 397.900
Cantidad media de iteraciones hasta contagio total con 750 personas: 245.800
Cantidad media de iteraciones hasta contagio total con 1000 personas: 161.100


Se puede observar que cuando se activa la opcion de "correlated walking" el contagio de la totalidad de las personas se produce considerablemente mas rapido, esto es debido a que las personas tienden por momentos a desplazarse mayores distancias en determinados sentidos, lo que provoca el acercamiento con otras personas y por lo tanto aumenta la probabilidad de contagio.

# Ejercicios 9 y 10

Estos 2 ejercicios estan hechos sobre el codigo anterior, introduciendo las correspondientes modificaciones.

In [29]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math as mt

const_N = 1000 # Cantidad de personas
const_proporcion_contagiados = 0.02 # Porcentaje de contagiados iniciales
const_tamanio_mapa = 200 # Tamanio del mapa
const_instantes_de_tiempo = 1000 # Instantes simulados

# Esto en "True" es el ej 8
correlated_random_walking = True

# Se puede observar que cuando se activa la opcion de correlated walking el contagio de la totalidad de las
# personas se produce considerablemente mas rapido, esto es debido a que las personas tienden por momentos
# a desplazarse mayores distancias en determinados sentidos, lo que provoca el acercamiento con otras personas
# y por lo tanto aumenta la probabilidad de contagio.

const_prob_cercana_contagio = 0.7   # celdas <= 3
const_prob_lejana_contagio = 0.5   # 3 < celdas <= 6




const_hay_re_contagio = True
const_alpha = 50
const_beta = 0.20

# Esto distinto de "1" es el ej 10
const_porcentaje_personas_moviles = 0.30

# Probabilidad de contagio del ej 10
# const_prob_cercana_contagio = 0.25   # celdas <= 3
# const_prob_lejana_contagio = 0.4   # 3 < celdas <= 6


const_up = 0
const_down = 1
const_right = 2
const_left = 3

class Persona:

  def __init__(self, tipo_de_persona, esta_enfermo, es_persona_movil, pos_x, pos_y):
    turnos = {"A":1, "B":2, "C":4}
    self._turnos_de_espera = turnos[tipo_de_persona]
    self._esta_enfermo = esta_enfermo
    self._turnos_esperados = 0
    self._pos_x = pos_x
    self._pos_y = pos_y
    self._movimiento_anterior = None
    self._se_contagio = esta_enfermo # (En algun momento)
    self._turnos_enfermo = 0
    self._es_persona_movil = es_persona_movil

  def intentar_moverse(self):
        self._turnos_esperados += 1

        if self._turnos_esperados == self._turnos_de_espera:
            self._turnos_esperados = 0
            return True

        return False

  # Con probabilidad "probabilidad_de_contagio" la persona se puede enfermar
  def intentar_contagiar(self, probabilidad_de_contagio):
    if (not self._esta_enfermo and (const_hay_re_contagio or (not self._se_contagio))):
      if rd.uniform(0, 1) <= probabilidad_de_contagio:
        self._esta_enfermo = True
        self._se_contagio = True
        
  # Si pasaron alpha turnos enfermo, con probabilidad beta la persona puede sanar
  def intentar_sanar(self):
    if (self._esta_enfermo):
        self._turnos_enfermo += 1
        if (self._turnos_enfermo > const_alpha):
            if (rd.uniform(0, 1) < const_beta):
                self._esta_enfermo = False
        
    else:
        self._turnos_enfermo = 0
    
    
  def esta_enferma(self):
    return (self._esta_enfermo)

  def obtener_posicion(self):
    return self._pos_x, self._pos_y

  def modificar_posicion(self, x, y):
    self._pos_x = x
    self._pos_y = y
    
  def obtener_movimiento_anterior(self):
    return self._movimiento_anterior

  def modificar_ultimo_movimiento(self, movimiento):
    self._movimiento_anterior = movimiento
    
  def es_persona_movil(self):
    return self._es_persona_movil


class Pandemia:

  def __init__(self):
    self.filas = const_tamanio_mapa
    self.columnas = const_tamanio_mapa
    self.mapa = generar_mapa(const_tamanio_mapa, const_tamanio_mapa)
    self.personas = poblar_mapa(self.mapa, const_N)
    self.lista_instantes = []
    self.lista_contagiados = []
    self.lista_sanos = []
    self.instantes_pasados = 0

  def info_personas(self):
      posicion_x = []
      posicion_y = []
      estado = []

      for persona in self.personas:
          x, y = persona.obtener_posicion()
          posicion_y.append(y)
          posicion_x.append(x)
            
          if (not persona.es_persona_movil()) and (not persona.esta_enferma()):
              estado.append('black')
          elif (not persona.es_persona_movil()) and persona.esta_enferma():
              estado.append('orange')
          elif persona.esta_enferma():
              estado.append('red')
          else:
              estado.append('green')

      return posicion_x, posicion_y, estado

  def siguiente_instante(self):
    self.lista_instantes.append(self.instantes_pasados)
    contagiados, sanos = estado_personas(self.personas)
    self.lista_contagiados.append(contagiados)
    self.lista_sanos.append(sanos)

    for persona in self.personas:

        if persona.esta_enferma():
            contagiar_cercanos(self.mapa, persona)

        if persona.intentar_moverse() and persona.es_persona_movil():
            mover_persona(self.mapa, persona)
            
        persona.intentar_sanar()

    self.instantes_pasados += 1



  def simular(self):
      # Sacado de un ejemplo de la documentacion de animation de pyplot
      #---------------------------------------
    
      fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(10, 10))
      ax.set_title("Simulación de contagios en una pandemia")
      ax.set_xlabel('Celdas (0.5 Metros)')
      ax.set_ylabel('Celdas (0.5 Metros)')
      personas = plt.scatter([], [])
      rect = plt.Rectangle([0, 0], 200, 200, ec='none', lw=1, fc='none')
      #---------------------------------------

      def animate(i):
          nonlocal rect, fig, personas
          ax.set_title("Simulación de contagios en una pandemia ( Instante Nro " + str(self.instantes_pasados) + " )")
          self.siguiente_instante()
          ms = 50 # Tamanio de los puntos
          rect.set_edgecolor('black')
          info_personas = self.info_personas()
          personas.remove()
          personas = plt.scatter(info_personas[0], info_personas[1], c=info_personas[2], s=ms)
          return personas, rect

      # Ejecuta "const_instantes_de_tiempo" veces la funcion "animate" con la que crea cada frame de la animacion
      ani = animation.FuncAnimation(fig, animate, interval=1, blit=True, frames=const_instantes_de_tiempo, repeat=False)

      ani.save('animation.mp4', fps=15)
    
      plt.close()
      plt.plot(self.lista_instantes, self.lista_contagiados)
      plt.xlabel("Instantes")
      plt.ylabel("Numero de contagiados")
      plt.title("Contagiados en funcion del tiempo")
      plt.savefig("Contagiados en funcion del tiempo.png")
      plt.clf()
      plt.plot(self.lista_instantes, self.lista_sanos)
      plt.xlabel("Instantes")
      plt.ylabel("Numero de sanos")
      plt.title("Sanos en funcion del tiempo")
      plt.savefig("Sanos en funcion del tiempo.png")


def generar_mapa(filas, columnas):
    mapa = []
    for i in range(filas):
        mapa.append([])
        for j in range(columnas):
            mapa[i].append(None)
    return mapa

def contagiar_cercanos(mapa, persona):
    x, y = persona.obtener_posicion()
    
    for i in range(-6, 7): # Rango [-6, 6]
        for j in range(-6, 7): # Rango [6, 6]
            pos_x = x + i
            pos_y = y + j
            
            # Utilizo la distancia Manhattan (El enunciado no especifica)
            
            if (posicion_es_valida(mapa, pos_x, pos_y)) and (mapa[pos_x][pos_y] is not None):
                if (((abs(i) + abs(j)) <= 6) and (abs(i) + abs(j)) > 3):
                    mapa[pos_x][pos_y].intentar_contagiar(const_prob_lejana_contagio)
                elif (abs(i) + abs(j) <= 3):
                    mapa[pos_x][pos_y].intentar_contagiar(const_prob_cercana_contagio)
                
            
def estado_personas(lista_personas):
    contagiados = 0
    sanos = 0
    for persona in lista_personas:
        if (persona.esta_enferma()):
            contagiados += 1
        else:
            sanos += 1
    return contagiados, sanos

def poblar_mapa(mapa, cantidad_de_gente):

    cupos_restantes = cantidad_de_gente
    cantidad_A = int(cantidad_de_gente * 0.70)
    cantidad_B = int(cantidad_de_gente * 0.25)
    cantidad_C = cantidad_de_gente - (cantidad_A + cantidad_B)
    cantidades = {"A":cantidad_A, "B":cantidad_B, "C":cantidad_C}
    return crear_personas(mapa, cantidades)

def crear_personas(mapa, cantidades):
    lista_personas = []
    contagiados_generados = 0
    personas_moviles_generadas = 0
    
    cantidad_de_gente = cantidades["A"] + cantidades["B"] + cantidades["C"]
    contagiados_totales = int(cantidad_de_gente * const_proporcion_contagiados)
    
    personas_moviles_total = int(cantidad_de_gente * const_porcentaje_personas_moviles)

    probabilidad_contagiado = const_proporcion_contagiados
    probabilidad_movil = const_porcentaje_personas_moviles

    for k in range(cantidad_de_gente):
        esta_contagiado = rd.uniform(0, 1) <= probabilidad_contagiado
        if (esta_contagiado):
            contagiados_generados += 1
            
        # Si ya se generaron todos los contagiados dejo de generarlos
        if (contagiados_generados == contagiados_totales):
            probabilidad_contagiado = -1
            
        # Si la cantidad de contagiados que faltan generar es igual a la cantidad de personas que faltan generar genero solo contagiados
        elif ((contagiados_totales - contagiados_generados) == (cantidad_de_gente - k - 1)):
            probabilidad_contagiado = 1
            
            
        es_persona_movil = rd.uniform(0, 1) <= probabilidad_movil
        if (es_persona_movil):
            personas_moviles_generadas += 1
            
        # Si ya se generaron todas las personas moviles dejo de generarlas
        if (personas_moviles_generadas == personas_moviles_total):
            probabilidad_movil = -1
            
        # Si la cantidad de personas moviles que faltan generar es igual a la cantidad de personas que faltan generar genero solo moviles
        elif ((personas_moviles_total - personas_moviles_generadas) == (personas_moviles_total - k - 1)):
            probabilidad_movil = 1   
            
            
        pos_x, pos_y = posicion_libre(mapa)
        persona = persona_random(cantidades, esta_contagiado, es_persona_movil, pos_x, pos_y)
        
        lista_personas.append(persona)
        mapa[pos_x][pos_y] = persona
    return lista_personas
    

#Devuelve dos enteros que representan una posicion aleatoria en el mapa que no se encuentra ocupada.
def posicion_libre(mapa):
    pos_x, pos_y = rd.randint(1, len(mapa) - 1), rd.randint(1, len(mapa[0]) - 1)
    while(not (mapa[pos_x][pos_y] is None)):
        pos_x, pos_y = rd.randint(1, len(mapa) - 1), rd.randint(1, len(mapa[0]) - 1)
    return pos_x, pos_y

def persona_random(cantidades, esta_enferma, es_persona_movil, pos_x, pos_y):
    cantidad_actual = cantidades["A"] + cantidades["B"] + cantidades["C"]
    rand_num = rd.randint(1, cantidad_actual)
    tipo = 0

    if (rand_num <= cantidades["A"]):
        tipo = "A"
    elif ((rand_num > cantidades["A"]) and (rand_num <= cantidades["A"] + cantidades["B"])):
        tipo = "B"
    else:
        tipo = "C"

    cantidades[tipo] -= 1
    return Persona(tipo, esta_enferma, es_persona_movil, pos_x, pos_y)

#Verifica si la posicion esta libre (No hay ninguna persona en esa posicion)
def posicion_esta_libre(mapa, pos_x, pos_y):
    return mapa[pos_x][pos_y] is None

#Verifica si la posicion es valida (Si la posicion se encuentra en el rango de valores que conforman los limites de la mapa).
def posicion_es_valida(mapa, pos_x, pos_y):
    return (pos_x >= 0) and (pos_y >= 0) and (pos_x < len(mapa)) and (pos_y < len(mapa[0]))

def repetir_movimiento(mapa, persona, pos_x, pos_y):
    se_movio = False
    
    if (persona.obtener_movimiento_anterior() == const_up):
        if ((posicion_es_valida(mapa, pos_x, pos_y + 1)) and (posicion_esta_libre(mapa, pos_x, pos_y + 1))):
            persona.modificar_posicion(pos_x, pos_y + 1)
            se_movio = True
        
    elif (persona.obtener_movimiento_anterior() == const_down):
        if ((posicion_es_valida(mapa, pos_x, pos_y - 1)) and (posicion_esta_libre(mapa, pos_x, pos_y - 1))):
            persona.modificar_posicion(pos_x, pos_y - 1)
            se_movio = True
        
    elif (persona.obtener_movimiento_anterior() == const_left):
        if ((posicion_es_valida(mapa, pos_x - 1, pos_y)) and (posicion_esta_libre(mapa, pos_x - 1, pos_y))):
            persona.modificar_posicion(pos_x - 1, pos_y)
            se_movio = True
            
    elif (persona.obtener_movimiento_anterior() == const_right):
        if ((posicion_es_valida(mapa, pos_x + 1, pos_y)) and (posicion_esta_libre(mapa, pos_x + 1, pos_y))):
            persona.modificar_posicion(pos_x + 1, pos_y)
            se_movio = True

    return se_movio

    
def mover_persona(mapa, persona):
    pos_x, pos_y = persona.obtener_posicion()
    se_movio = False
    
    # Con probabilidad 0.4 se repite el movimiento anterior
    if ((correlated_random_walking == True) and (rd.uniform(0, 1) < 0.4) and (persona.obtener_movimiento_anterior() is not None)) : # Repito direccion y sentido
        se_movio = repetir_movimiento(mapa, persona, pos_x, pos_y)
        
    #La probabilidad de moverse en direccion horizontal o verticalmente es del 0.5 para cada caso.
    #De igual manera, la probabilidad de moverse en cada sentido es de 0.5 para cada caso.
    #No se permiten movimientos en diagonal.
    
    
    # Obtengo un nuevo movimiento aleatorio (Puede repetirse)
    elif rd.uniform(0, 1) < 0.5: #Movimiento Horizontal

        if (rd.uniform(0, 1) < 0.5) and (posicion_es_valida(mapa, pos_x, pos_y + 1)) and (posicion_esta_libre(mapa, pos_x, pos_y + 1)):

            persona.modificar_posicion(pos_x, pos_y + 1)
            persona.modificar_ultimo_movimiento(const_up)
            se_movio = True
            

        elif (posicion_es_valida(mapa, pos_x, pos_y - 1)) and (posicion_esta_libre(mapa, pos_x, pos_y - 1)):

            persona.modificar_posicion(pos_x, pos_y - 1)
            persona.modificar_ultimo_movimiento(const_down)
            se_movio = True

    else: #Movimiento Vertical

        if (rd.uniform(0, 1) < 0.5) and (posicion_es_valida(mapa, pos_x - 1, pos_y)) and (posicion_esta_libre(mapa, pos_x - 1, pos_y)):

            persona.modificar_posicion(pos_x - 1, pos_y)
            persona.modificar_ultimo_movimiento(const_left)
            se_movio = True

        elif (posicion_es_valida(mapa, pos_x + 1, pos_y)) and (posicion_esta_libre(mapa, pos_x + 1, pos_y)):

            persona.modificar_posicion(pos_x + 1, pos_y)
            persona.modificar_ultimo_movimiento(const_right)
            se_movio = True

    if se_movio:
        x, y = persona.obtener_posicion()
        mapa[x][y] = persona
        mapa[pos_x][pos_y] = None
        

#### Ejercicio 9
Ahora las personas tienen una chance de curarse luego de pasar una cantidad determinada de tiempo. Utilizando los valores alpha = 50 y beta = 0.2. En este caso no vamos a calcular hasta contagio total porque nunca va a ocurrir o tarda demasiado. Se pueden ver los gráficos y animaciones en el drive

Un escenario particular que se da en estos casos es cuando no hay re-contagios donde se puede observar en el grafico de contagiados un pico que crece hasta determinado valor para luego decrecer hasta cero. En la situacion en la que hay re-contagios lo que se observa es que las personas se estan enfermando continuamente y el virus nunca termina de desaparecer.


#### Ejercicio 10
Como última modificación vamos a ver que ocurre cuando un porcentaje de la población no se mueve. Decidimos usar un valor de lambda = 0.3.

Lo observado es similar al ejercicio 9 cuando el porcentaje de personas moviles es cercano a uno. Por otro lado, al disminuir las probabilidades de contagio y el numero de personas que se mueven los contagios se producen mas lentamente.